In [122]:
#!pip3 install langchain # !pip install pytesseract # pip install requests
# !pip install tiktoken -q
# !pip install unstructured
# !pip install "unstructured[pdf]"
# !pip install partition-pdf
!pip install --force-reinstall -v "unstructured==0.7.12"

Using pip 23.3.1 from /Users/srikanthsrinivasan/Documents/litpal/.venv/lib/python3.11/site-packages/pip (python 3.11)
  Obtaining dependency information for unstructured==0.7.12 from https://files.pythonhosted.org/packages/a4/91/80564119abc4339a775c6f9c3f2085e69d98a1122c8670d8ba562b5fe1d2/unstructured-0.7.12-py3-none-any.whl.metadata
  Using cached unstructured-0.7.12-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for argilla from https://files.pythonhosted.org/packages/8f/d8/2af275c2a8df0d8e06a8d7daa8231431b2b1fd99f2d2169a131f41ff0d58/argilla-1.18.0-py3-none-any.whl.metadata
  Using cached argilla-1.18.0-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Obtaini

In [89]:
from bs4 import BeautifulSoup
import requests
import pytesseract
from PIL import Image

from pdf2image import convert_from_path

# define headers argument
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

In [34]:
# query

search = "Attention is all you need"

# format the google scholar article link
# 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=web+scraping&btnG='
base = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q='
end = '&btnG='
url = base + '+'.join(search.split()) + end

response=requests.get(url,headers=headers)

# soup query
soup=BeautifulSoup(response.content, "html.parser") # 'lxml')

In [78]:
#print(soup)

dict = []

for item in soup.select('[data-lid]'):
	try:
		add = {}
		# print('----------------------------------------')

		# title
		print(item.select('h3')[0].get_text())
		add['title'] = item.select('h3')[0].get_text()

		# URL download link for the PDF
		# print(item.select('a')[0]['href'])
		add['url'] = item.select('a')[0]['href']

		print('----------------------------------------')

		# print(add)
		
		dict.append(add)
	except Exception as e:
		#raise e
		print('')

print(dict)

Attention is all you need
… to attend to all positions in the decoder up to and including that position. We need to prevent 
… We implement this inside of scaled dot-product attention by masking out (setting to −∞) …
190
----------------------------------------
Attention is all you need in speech separation
… Transformers are emerging as a natural alternative to standard RNNs, replacing recurrent 
computations with a multi-head attention mechanism.In this paper, we propose the SepFormer…
183
----------------------------------------
Attention is all you need
… Attention • Visual attention and textual attention … Could we build a transformer-based 
model whose language model looks both forward and backwards, ie “is conditioned on both …
180
----------------------------------------
Channel attention is all you need for video frame interpolation
… Since every spatial region is important for pixel-level synthesis, we do not explicitly consider 
spatial attention. Instead, we adopt the chann

In [86]:
# download the pdf

url = dict[0]['url']

# url = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD"
query_parameters = {"downloadformat": "pdf"}

response = requests.get(url, params=query_parameters)

with open("test.pdf", mode="wb") as file:
    file.write(response.content)


In [99]:
# OCR up the pdf

pages = convert_from_path('test.pdf', 500)
    
# Extract text from each page using Tesseract OCR
text_data = ''
for page in pages:
    text = pytesseract.image_to_string(page)
    text_data += text + '\n'

# Return the text data
print(text_data)

 

Attention Is All You Need

Ashish Vaswani* Noam Shazeer* Niki Parmar* Jakob Uszkoreit*
Google Brain Google Brain Google Research Google Research
avaswani@google.com noam@google.com nikip@google.com usz@google.com

Llion Jones* Aidan N. Gomez* ' Lukasz Kaiser*
Google Research University of Toronto Google Brain
llion@google.com aidan@cs.toronto.edu lukaszkaiser@google.com

Illia Polosukhin* #

illia.polosukhin@gmail.com

Abstract

The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train.

In [100]:
file = open('file.txt', 'w')
file.write(text_data)
file.close()

In [107]:
import os
import openai
import pinecone

from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

/Users/srikanthsrinivasan/Documents/litpal/.venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [125]:
directory = "contents/"

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

ImportError: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"

In [ ]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 20

def split_docs(documents, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

In [ ]:
embeddings = OpenAIEmbeddings(model_name="ada")

query_result = embeddings.embed_query("Hello world")
len(query_result)

In [ ]:
# pinecone database

pinecone.init(
    api_key="c5e1f4f4-e66b-4ae5-b500-e60b43dd5c84",
    environment="env"
)

index_name = "langchain-demo"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
def get_similiar_docs(query, k=2, score=False):  # we can control k value to get no. of context with respect to question.
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs


from langchain.llm import AzureOpenAI


model_name = "text-davinci-003"
llm = AzureOpenAI(model_name=model_name)

chain = load_qa_chain(llm, chain_type="stuff") #we can use map_reduce chain_type also.

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  print(similar_docs)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [ ]:
query = ""
answer = get_answer(query)
print(answer)